In [1]:
print("Guys lets do this")

Guys lets do this


In [32]:
import pickle

my_variable = {'user_id': 1, 'top_k': 5}
with open('shared_data.pkl', 'wb') as f:
    pickle.dump(my_variable, f)
print("done")

done


In [1]:
import re
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
movies=pd.read_csv('movies.csv')
ratings_chunks = pd.read_csv('ratings.csv', chunksize=100000)
ratings = pd.concat(ratings_chunks)
users=pd.read_csv('users.csv')

# Display basic info
print("Movies:\n",movies.info())
print("\nRatings:\n",ratings.info())
print("\nUsers:\n",users.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3706 entries, 0 to 3705
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie_id      3706 non-null   int64 
 1   movie_title   3706 non-null   object
 2   movie_genres  3706 non-null   object
 3   poster_url    3648 non-null   object
dtypes: int64(1), object(3)
memory usage: 115.9+ KB
Movies:
 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   user_id      1000209 non-null  int64  
 1   movie_id     1000209 non-null  int64  
 2   user_rating  1000209 non-null  float64
 3   timestamp    1000209 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 30.5 MB

Ratings:
 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 6 columns):
 #   Column                

In [3]:
# Download necessary resources
nltk.download('punkt_tab')

# Drop duplicates and missing values
movies.drop_duplicates(inplace=True)
movies.dropna(inplace=True)

# Split genres into lists
movies['movie_genres'] = movies['movie_genres'].apply(lambda x: x.split('|'))

# One-hot encode genres
mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(movies['movie_genres']), columns=mlb.classes_)
movies = pd.concat([movies.drop('movie_genres', axis=1), genres_encoded], axis=1)
movies['movie_title'] = movies['movie_title'].astype(str)
# Clean and tokenize movie_movie_titles
def clean_movie_movie_title(movie_movie_title):
    return re.sub(r'\(\d{4}\)', '', movie_movie_title).strip()

movies['clean_movie_title']=movies['movie_title'].apply(clean_movie_movie_title)
movies['movie_title_tokens']=movies['clean_movie_title'].apply(word_tokenize)
movies['movie_title_token_count']=movies['movie_title_tokens'].apply(len)
print("done")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\RGUKTS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


done


In [4]:
# Drop duplicates and missing values
ratings.drop_duplicates(inplace=True)
ratings.dropna(inplace=True)

# Convert timestamp to datetime
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

# Extract date features
ratings['year'] = ratings['timestamp'].dt.year
ratings['month'] = ratings['timestamp'].dt.month
ratings['day'] = ratings['timestamp'].dt.day

# Filter users and movies with too few ratings
user_counts = ratings['user_id'].value_counts()
movie_counts = ratings['movie_id'].value_counts()

ratings = ratings[
    ratings['user_id'].isin(user_counts[user_counts >= 10].index) &
    ratings['movie_id'].isin(movie_counts[movie_counts >= 10].index)
]

print(ratings['user_rating'].describe())

print("done")

count    998539.000000
mean          3.582575
std           1.116516
min           1.000000
25%           3.000000
50%           4.000000
75%           4.000000
max           5.000000
Name: user_rating, dtype: float64
done


In [6]:
# Drop duplicates and missing values
users.drop_duplicates(inplace=True)
users.dropna(inplace=True)

# Convert to categorical
users['gender'] = users['user_gender'].astype('category')
users['occupation'] = users['user_occupation_label'].astype('category')

# Encode gender and occupation
users['gender_encoded'] = LabelEncoder().fit_transform(users['gender'])
users['occupation_encoded'] = LabelEncoder().fit_transform(users['occupation'])
print("done")

done


In [7]:
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
print("\n=== Basic Dataset Information ===")
print("-" * 50)
print(f"Number of movies: {len(movies)}")
print(f"Number of ratings: {len(ratings)}")
print(f"Number of users: {len(users)}")


=== Basic Dataset Information ===
--------------------------------------------------
Number of movies: 3701
Number of ratings: 998539
Number of users: 6040


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter out rare genres (e.g., with less than 20 movies)
filtered_genre_counts = genre_counts[genre_counts > 20]

# Sort genres again (optional)
filtered_genre_counts = filtered_genre_counts.sort_values(ascending=True)

# Set a clean style
sns.set(style="whitegrid")

# Plot
plt.figure(figsize=(10, 8))
sns.barplot(x=filtered_genre_counts.values, y=filtered_genre_counts.index)

plt.movie_movie_title('Top Genres by Number of Movies', fontsize=14)
plt.xlabel('Number of Movies', fontsize=12)
plt.ylabel('Genre', fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.tight_layout()
plt.savefig('clean_genre_distribution.png')
plt.show()


In [ ]:
# ==========================
# ✅ RATINGS ANALYSIS
# ==========================
print("\n=== Ratings Analysis ===")
print("-" * 50)

# Distribution of ratings
print("Creating ratings distribution plot...")
plt.figure(figsize=(10, 6))
sns.histplot(data=ratings, x='user_rating', bins=20)
plt.movie_movie_title('Distribution of Movie Ratings')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.savefig('ratings_distribution.png')
plt.show()

# Average rating per movie
print("Calculating average ratings per movie...")
movie_ratings = ratings.groupby('movie_id')['user_rating'].agg(['mean', 'count']).reset_index()
movie_ratings = movie_ratings.merge(movies[['movie_id', 'movie_movie_movie_title']], on='movie_id', how='left')

print("\nTop 10 Highest Rated Movies (with at least 100 ratings):")
print(movie_ratings[movie_ratings['count'] >= 100].sort_values('mean', ascending=False).head(10))

In [ ]:
# ==========================
# ✅ USER BEHAVIOR ANALYSIS
# ==========================
print("\n=== User Behavior Analysis ===")
print("-" * 50)

# Number of ratings per user
print("Analyzing user rating patterns...")
user_ratings = ratings.groupby('user_id').size().reset_index(name='rating_count')

plt.figure(figsize=(10, 6))
sns.histplot(data=user_ratings, x='rating_count', bins=50)
plt.movie_movie_title('Distribution of Ratings per User')
plt.xlabel('Number of Ratings')
plt.ylabel('Number of Users')
plt.savefig('user_ratings_distribution.png')
plt.show()

print("\nUser Rating Statistics:")
print(user_ratings['rating_count'].describe())

# ==========================
# ✅ EXPORT ANALYSIS FILES
# ==========================
print("\nSaving analysis results...")
movie_ratings.to_csv('movie_ratings_analysis.csv', index=False)
user_ratings.to_csv('user_ratings_analysis.csv', index=False)
genre_df.to_csv('genre_analysis.csv', index=False)

print("\nAnalysis complete! Check the generated files:")
print("1. infotact_project1/genre_distribution.png")
print("2. infotact_project1/ratings_distribution.png")
print("3. infotact_project1/user_ratings_distribution.png")
print("4. infotact_project1/movie_ratings_analysis.csv")
print("5. infotact_project1/user_ratings_analysis.csv")
print("6. infotact_project1/genre_analysis.csv")

In [8]:
df=ratings.merge(users, on='user_id').merge(movies, on='movie_id')
print(df.head())

   user_id  movie_id  user_rating           timestamp  year  month  day  \
0      130      3107          5.0 2000-12-21 20:56:33  2000     12   21   
1     3829      2114          4.0 2000-08-10 18:42:47  2000      8   10   
2     1265       256          1.0 2002-01-27 03:52:32  2002      1   27   
3     2896      1389          5.0 2000-10-20 01:16:45  2000     10   20   
4     5264      3635          4.0 2000-06-16 18:28:31  2000      6   16   

   user_gender  bucketized_user_age  user_occupation_label  ... 9, 10 9, 13  \
0         True                 35.0                     18  ...   0.0   0.0   
1        False                 25.0                      0  ...   0.0   0.0   
2        False                 18.0                     21  ...   0.0   0.0   
3         True                 18.0                     14  ...   0.0   0.0   
4         True                 18.0                     17  ...   0.0   0.0   

  9, 13, 16 9, 14, 16  9, 15  9, 15, 16 9, 16      clean_movie_title  \
0 

In [14]:
# Import necessary libraries
from sklearn.metrics.pairwise import cosine_similarity

# Pivot to create user-item matrix
user_item = ratings.pivot_table(index='user_id', columns='movie_id', values='user_rating')
user_item = user_item.fillna(0)

# Compute cosine similarity between users
user_sim = cosine_similarity(user_item)
user_sim_df = pd.DataFrame(user_sim, index=user_item.index, columns=user_item.index)

# Predict ratings for a user based on similar users
def predict_ratings(target_user_id, user_item, user_sim_df, k=5):
    sim_scores = user_sim_df[target_user_id].drop(target_user_id)
    top_k_users = sim_scores.nlargest(k).index
    sim_subset = sim_scores[top_k_users]
    neighbor_ratings = user_item.loc[top_k_users]
    weighted_sum = (neighbor_ratings.mul(sim_subset, axis=0)).sum(axis=0)
    sim_sum = sim_subset.sum()
    predicted_ratings = weighted_sum / sim_sum
    return predicted_ratings

# Recommend top N movies that the user hasn't rated
def recommend_movies(target_user_id, user_item, user_sim_df, movies_df, k=5, n_recs=10):
    predicted = predict_ratings(target_user_id, user_item, user_sim_df, k)
    already_rated = user_item.loc[target_user_id] > 0
    predicted = predicted[~already_rated]
    top_movies = predicted.nlargest(n_recs).index
    recommendations = movies_df[movies_df['movie_id'].isin(top_movies)][['movie_id', 'movie_title']]
    recommendations = recommendations.set_index('movie_id').loc[top_movies].reset_index()
    return recommendations

# Example: Get top 5 recommendations for user ID 1
user_id = 1
recommendations = recommend_movies(user_id, user_item, user_sim_df, movies, k=10, n_recs=5)

print(f"Top 5 movie recommendations for User {user_id}:")
print(recommendations)


Top 5 movie recommendations for User 1:
   movie_id                 movie_title
0    2081.0  Little Mermaid, The (1989)
1    2078.0     Jungle Book, The (1967)
2    2096.0      Sleeping Beauty (1959)
3     364.0       Lion King, The (1994)
4    1282.0             Fantasia (1940)


In [15]:
# Import necessary libraries
from sklearn.metrics.pairwise import cosine_similarity

# Pivot to create item-user matrix (transpose of user-item)
item_user = ratings.pivot_table(index='movie_id', columns='user_id', values='user_rating')
item_user = item_user.fillna(0)

# Compute cosine similarity between items (movies)
item_sim = cosine_similarity(item_user)
item_sim_df = pd.DataFrame(item_sim, index=item_user.index, columns=item_user.index)

# Predict ratings for a user based on similar items
def predict_item_based_ratings(target_user_id, item_user, item_sim_df):
    user_ratings = item_user.loc[:, target_user_id]
    rated_items = user_ratings[user_ratings > 0].index

    predictions = {}
    for item in item_user.index:
        if user_ratings[item] == 0:
            sim_items = item_sim_df.loc[item, rated_items]
            sim_scores = sim_items.values
            ratings = user_ratings[rated_items].values
            if sim_scores.sum() != 0:
                predicted_rating = np.dot(sim_scores, ratings) / sim_scores.sum()
                predictions[item] = predicted_rating

    return pd.Series(predictions).sort_values(ascending=False)

# Recommend top N movies that the user hasn't rated
def recommend_movies_item_based(target_user_id, item_user, item_sim_df, movies_df, n_recs=10):
    predicted_ratings = predict_item_based_ratings(target_user_id, item_user, item_sim_df)
    top_items = predicted_ratings.head(n_recs).index
    recommendations = movies_df[movies_df['movie_id'].isin(top_items)][['movie_id', 'movie_title']]
    recommendations = recommendations.set_index('movie_id').loc[top_items].reset_index()
    return recommendations

# Example: Get top 5 recommendations for user ID 1
user_id = 1
recommendations = recommend_movies_item_based(user_id, item_user, item_sim_df, movies, n_recs=5)

print(f"Top 5 movie recommendations for User {user_id} (Item-Based):")
print(recommendations)


Top 5 movie recommendations for User 1 (Item-Based):
    index                                        movie_title
0  2913.0  Mating Habits of the Earthbound Human, The (1998)
1   860.0        Maybe, Maybe Not (Bewegte Mann, Der) (1994)
2  3058.0                                    Ape, The (1940)
3  2636.0                          Mummy's Ghost, The (1944)
4  3372.0                      Bridge at Remagen, The (1969)


In [21]:
from sklearn.model_selection import train_test_split

# Create a new DataFrame for train/test split
ratings_data = ratings[['user_id', 'movie_id', 'user_rating']]

# Train-test split by rating records
train_data, test_data = train_test_split(ratings_data, test_size=0.2, random_state=42)


In [22]:
from sklearn.decomposition import TruncatedSVD, NMF
# Fill missing ratings with 0s for simplicity (alternatively, you can try mean-centering)
train_matrix = train_data.pivot(index='user_id', columns='movie_id', values='user_rating').fillna(0)
user_item_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='user_rating').fillna(0)

# Define number of latent features
n_components = 50  # You can tune this
# SVD: user_item_matrix ≈ U * Sigma * V^T
svd = TruncatedSVD(n_components=n_components, random_state=42)
user_features_svd = svd.fit_transform(user_item_matrix)
item_features_svd = svd.components_

# Reconstruct the ratings matrix
predicted_ratings_svd = np.dot(user_features_svd, item_features_svd)

# Convert to DataFrame for easy interpretation
predicted_svd_df = pd.DataFrame(predicted_ratings_svd, index=user_item_matrix.index, columns=user_item_matrix.columns)

# Example: Get Top 5 recommendations for user 1
def recommend_svd(user_id, predicted_df, movies_df, n_recs=5):
    user_row = predicted_df.loc[user_id]
    rated_movies = user_item_matrix.loc[user_id] > 0
    user_row = user_row[~rated_movies]
    top_movie_ids = user_row.nlargest(n_recs).index
    return movies_df[movies_df['movie_id'].isin(top_movie_ids)][['movie_id', 'movie_title']]

print("\nTop 5 Recommendations using SVD:")
print(recommend_svd(1, predicted_svd_df, movies))



Top 5 Recommendations using SVD:
     movie_id                       movie_title
117     318.0  Shawshank Redemption, The (1994)
175    2081.0        Little Mermaid, The (1989)
346      34.0                       Babe (1995)
576     364.0             Lion King, The (1994)
957    1282.0                   Fantasia (1940)


In [24]:
nmf = NMF(n_components=1, init='random', random_state=42)
user_features_nmf = nmf.fit_transform(user_item_matrix)
item_features_nmf = nmf.components_

# Reconstruct the ratings
predicted_ratings_nmf = np.dot(user_features_nmf, item_features_nmf)

# Convert to DataFrame
predicted_nmf_df = pd.DataFrame(predicted_ratings_nmf, index=user_item_matrix.index, columns=user_item_matrix.columns)

# Example: Get Top 5 recommendations for user 1
def recommend_nmf(user_id, predicted_df, movies_df, n_recs=5):
    user_row = predicted_df.loc[user_id]
    rated_movies = user_item_matrix.loc[user_id] > 0
    user_row = user_row[~rated_movies]
    top_movie_ids = user_row.nlargest(n_recs).index
    return movies_df[movies_df['movie_id'].isin(top_movie_ids)][['movie_id', 'movie_title']]

print("\nTop 5 Recommendations using NMF:")
print(recommend_nmf(1, predicted_nmf_df, movies))



Top 5 Recommendations using NMF:
     movie_id                                        movie_title
154    2571.0                                 Matrix, The (1999)
215    2858.0                             American Beauty (1999)
257     593.0                   Silence of the Lambs, The (1991)
331    1198.0                     Raiders of the Lost Ark (1981)
859    1196.0  Star Wars: Episode V - The Empire Strikes Back...


In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_rmse_mae(test_data, pred_df, name="Model"):
    y_true, y_pred = [], []
    for _, row in test_data.iterrows():
        user, movie, rating = row['user_id'], row['movie_id'], row['user_rating']
        if user in pred_df.index and movie in pred_df.columns:
            y_true.append(rating)
            y_pred.append(pred_df.loc[user, movie])
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{name} RMSE: {rmse:.4f}")
    print(f"{name} MAE : {mae:.4f}")

evaluate_rmse_mae(test_data, predicted_svd_df, "SVD")
evaluate_rmse_mae(test_data, predicted_nmf_df, "NMF")


SVD RMSE: 2.2345
SVD MAE : 1.9075
NMF RMSE: 2.9457
NMF MAE : 2.6849


In [27]:
def precision_at_k(pred_df, train_df, test_df, k=10, threshold=3.5):
    precision_list = []
    for user in test_df['user_id'].unique():
        if user not in pred_df.index:
            continue
        user_rated_movies = set(train_df[train_df['user_id'] == user]['movie_id'])
        user_test = test_df[test_df['user_id'] == user]
        user_preds = pred_df.loc[user].drop(labels=user_rated_movies, errors='ignore')
        top_k_preds = user_preds.sort_values(ascending=False).head(k)
        
        relevant = set(user_test[user_test['user_rating'] >= threshold]['movie_id'])
        recommended = set(top_k_preds.index)
        hits = len(recommended & relevant)
        precision = hits / k
        precision_list.append(precision)
    return np.mean(precision_list)

print(f"SVD Precision@10: {precision_at_k(predicted_svd_df, train_data, test_data):.4f}")
print(f"NMF Precision@10: {precision_at_k(predicted_nmf_df, train_data, test_data):.4f}")


SVD Precision@10: 0.4143
NMF Precision@10: 0.1508


In [30]:
# Week 3 & 4: Content-Based, Hybrid Recommender, Evaluation Table, Streamlit UI

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

# --- Content-Based Filtering ---
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['clean_movie_title'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
title_to_index = pd.Series(movies.index, index=movies['clean_movie_title'].str.lower())

def get_content_recommendations(title, n=10):
    title = title.lower()
    if title not in title_to_index:
        return pd.DataFrame(columns=["movie_id", "movie_title"])
    idx = title_to_index[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    movie_indices = [i[0] for i in sim_scores]
    return movies[['movie_id', 'movie_title']].iloc[movie_indices].reset_index(drop=True)

# --- Collaborative Filtering with SVD ---
user_item_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='user_rating').fillna(0)
svd = TruncatedSVD(n_components=50, random_state=42)
user_features = svd.fit_transform(user_item_matrix)
item_features = svd.components_
predicted_ratings = np.dot(user_features, item_features)
predicted_df = pd.DataFrame(predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)

def recommend_svd(user_id, predicted_df, movies_df, user_item_df, n=5):
    if user_id not in predicted_df.index:
        return pd.DataFrame(columns=["movie_id", "movie_title"])
    user_row = predicted_df.loc[user_id]
    user_row = user_row[~(user_item_df.loc[user_id] > 0)]
    top_ids = user_row.nlargest(n).index
    recs = movies_df[movies_df['movie_id'].isin(top_ids)][['movie_id', 'movie_title']]
    return recs.set_index('movie_id').loc[top_ids].reset_index()

# --- Hybrid Recommender (SVD + Content Intersection) ---
def hybrid_recommend(user_id, predicted_df, movies_df, user_item_df, tfidf_sim=cosine_sim, n=5):
    if user_id not in predicted_df.index:
        return pd.DataFrame(columns=["movie_id", "movie_title"])
    user_row = predicted_df.loc[user_id]
    unrated = user_row[~(user_item_df.loc[user_id] > 0)]
    top_svd_ids = unrated.nlargest(30).index  # wider pool
    movie_idxs = movies[movies['movie_id'].isin(top_svd_ids)].index
    sim_avg = cosine_sim[movie_idxs].mean(axis=1)
    top_idx = sim_avg.argsort()[::-1][:n]
    movie_ids = movies.iloc[movie_idxs[top_idx]]['movie_id']
    return movies[movies['movie_id'].isin(movie_ids)][['movie_id', 'movie_title']].reset_index(drop=True)

# --- Evaluation Table Example (dummy values) ---
evaluation_results = pd.DataFrame({
    'Model': ['User-Based CF', 'Item-Based CF', 'SVD', 'Content-Based'],
    'RMSE': [0.96, 0.94, 0.91, None],
    'Precision@10': [0.21, 0.23, 0.26, 0.18]
})
print("\nModel Evaluation Summary:")
print(evaluation_results)



Model Evaluation Summary:
           Model  RMSE  Precision@10
0  User-Based CF  0.96          0.21
1  Item-Based CF  0.94          0.23
2            SVD  0.91          0.26
3  Content-Based   NaN          0.18
